Untyped terms feels kind of like a toy. It is super useful for tinkering and if you know what you're doing it is less constrained. But, it is way easier to make mistakes in your formulation
https://tptp.org/TPTP/SyntaxBNF.html

In [1]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Sort:
    pass

@dataclass(frozen=True)
class TConst(Sort):
    name: str
    args: tuple[Sort] = ()

IntSort = TConst('$int')
BoolSort = TConst('$o')
RealSort = TConst('$real')
RatSort = TConst('$rat')



In [ ]:
@dataclass(frozen=True)
class Term:
    def __eq__(self : Term, other : Term):
        return Eq(self, other)

@dataclass(frozen=True)
class Var(Term):
    name: str
    ty : Sort
    def __repr__(self):
        return self.name.title()
    def vars(self):
        return {self}

@dataclass(frozen=True)
class Fn(Term):
    name: str
    ty: Sort
    args: tuple[Term, ...]
    def __eq__(self : Term, other : Term):
        return Eq(self, other)
    def vars(self):
        return {v for a in self.args for v in a.vars()}
    def __repr__(self):
        return f"{self.name}({', '.join(map(str, self.args))})"

@dataclass(frozen=True)
class InFix(Fn):
    name: str
    ty : Sort


class Lambda(Term):
    args: Tuple[Var, ...]
    body: Term
    def __repr__(self):
        pass

In [ ]:
def Const(name, ty):
    return Fn(name, (), ty)
def Consts(names):
    return [Const(name) for name in names.split()]

We can really bulk out our special formula constructors


In [ ]:
@dataclass(frozen=True)
class Formula:
    pass

@dataclass(frozen=True)
class FTrue(Formula):
    def thf(self):
        return "$true"

@dataclass(frozen=True)
class FFalse(Formula):
    def thf(self):
        return "$false"
    

@dataclass(frozen=True)
class Not(Formula):
    arg: Formula
    def thf(self):
        return f"~({self.arg.thf()})"

@dataclass(frozen=True)
class And(Formula):
    x: Formula
    y: Formula
    def thf(self):
        return f"({self.x.thf()} & {self.y.thf()})"

@dataclass(frozen=True)
class Or(Formula):
    x: Formula
    y: Formula
    def thf(self):
        return f"({self.args[0].thf()} | {self.args[1].thf()})"

@dataclass(frozen=True)
class Implies(Formula):
    hyp: Formula
    conc: Formula
    def thf(self):
        return f"({self.hyp.thf()} => {self.conc.thf()})"

@dataclass(frozen=True)
class Iff(Formula):
    lhs: Formula
    rhs: Formula
    def thf(self):
        return f"({self.lhs.thf()} <=> {self.rhs.thf()})"

@dataclass(frozen=True)
class ForAll(Formula):
    var: str
    body: Formula
    def thf(self):
        return f"! [ {self.var} ] : ({self.body.thf()})"

@dataclass(frozen=True)
class Exists(Formula):
    var: str
    body: Formula
    def thf(self):
        return f"? [ {self.var} ] : ({self.body.thf()})"

@dataclass(frozen=True)
class Atom(Formula):
    pred: str
    args: tuple[Term]


@dataclass(frozen=True)
class Eq(Formula):
    lhs: Term
    rhs: Term
    def thf(self):
        return f"({self.lhs.thf()} = {self.rhs.thf()})"



In [ ]:
class ProofDB():
    def __init__(self):
        self.__proof_db = []
        self.defs = {}
        self.types = {}
    def make_type(self,name):
        t = TConst(name)
        self.types[name] = t
        return t
    def make_const(self,name,ty):
        c = Const(name,ty)
        self.defs[name] = c
        return c

        